In [1]:
sc.version

'2.4.5'

In [7]:
#from pyspark import SparkContext
#sc = SparkContext.getOrCreate()
#sqlContext = SQLContext(sc)

df = sqlContext.read.format('csv').options(header='true', inferschema='true')\
    .load('../Iris.csv')
df.rdd.take(3)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa'),
 Row(Id=2, SepalLengthCm=4.9, SepalWidthCm=3.0, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa'),
 Row(Id=3, SepalLengthCm=4.7, SepalWidthCm=3.2, PetalLengthCm=1.3, PetalWidthCm=0.2, Species='Iris-setosa')]

In [8]:
hostname = "127.0.0.1" 
dbname = "farmadb"
jdbcPort = 3306
username = "root"
password = "12345"
jdbc_url = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}"\
            .format(hostname, jdbcPort, dbname, username, password)

query = "(select * from FAMILIA) t1_alias"
df1 = sqlContext.read.format('jdbc')\
    .options(driver = 'com.mysql.jdbc.Driver', url=jdbc_url, dbtable=query)\
    .load()
df1.show()
#df1.toPandas()

+------+----------+
|Fam_id|   Nom_fam|
+------+----------+
|     1|   TABLETA|
|     2|INYECTABLE|
|     3|SUSPENSION|
+------+----------+



In [29]:
#Feature selection
df2=df.select('SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species')
#df2.show()

In [30]:
drop_list = ['Id'] # Remove Id column
df2 = df.select([column for column in df.columns if column not in drop_list])
df2.show()

+-------------+------------+-------------+------------+-----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+-------------+------------+-------------+------------+-----------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
|          5.4|         3.7|          1.5|         0.2|Iris-setosa|
|          4.8|         3.4|          1.6|      

In [31]:
df2.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [32]:
df2.groupBy("Species").count().show()

+---------------+-----+
|        Species|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   50|
|Iris-versicolor|   50|
+---------------+-----+



In [35]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

stringIndexer = StringIndexer(inputCol = "Species", outputCol = "label")

# Solo para mostrar
modelx = stringIndexer.fit(df2)
indexedx = modelx.transform(df2)
#type(modelx)

In [36]:
indexedx.toPandas()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label
0,5.1,3.5,1.4,0.2,Iris-setosa,0.0
1,4.9,3.0,1.4,0.2,Iris-setosa,0.0
2,4.7,3.2,1.3,0.2,Iris-setosa,0.0
3,4.6,3.1,1.5,0.2,Iris-setosa,0.0
4,5.0,3.6,1.4,0.2,Iris-setosa,0.0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica,2.0
146,6.3,2.5,5.0,1.9,Iris-virginica,2.0
147,6.5,3.0,5.2,2.0,Iris-virginica,2.0
148,6.2,3.4,5.4,2.3,Iris-virginica,2.0


In [37]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features")

dfa = vecAssembler.transform(df2) # this will add to the table outputCol column with vectors.
dfa.show()# dfa ya no es necesario, ahora se encarga Pipeline

+-------------+------------+-------------+------------+-----------+-----------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|         features|
+-------------+------------+-------------+------------+-----------+-----------------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
|          5.4|         3.9|          1.7|         0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|
|          4.6|         3.4|          1.4|         0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|
|          5.0|         3.4|          1.5|         0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|
|          4.4|         2.9|          1.4|         0.2

In [38]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssembler, stringIndexer])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df2)
dataset = pipelineFit.transform(df2)
dataset.show(5)

+-------------+------------+-------------+------------+-----------+-----------------+-----+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|         features|label|
+-------------+------------+-------------+------------+-----------+-----------------+-----+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|  0.0|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|  0.0|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|  0.0|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|  0.0|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|  0.0|
+-------------+------------+-------------+------------+-----------+-----------------+-----+
only showing top 5 rows



In [39]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 103
Test Dataset Count: 47


In [43]:
#trainingData.show()

In [42]:
#testData.show()

In [ ]:
from pyspark.ml.classification import NaiveBayes
